# Pre set up

In [1]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import cv2
import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog,MetadataCatalog
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 10,10
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    plt.show()

# Train ORCNN on amodal datasets

## Register my amodal datasets 

In [2]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
dataDir='datasets/coco'
annTrainFile='{}/annotations/COCO_amodal_train2014_with_classes_poly.json'.format(dataDir)
imgTrainFile = '{}/train2014'.format(dataDir)
register_coco_instances("amodal_coco_train", {},annTrainFile , imgTrainFile)
# Prepare test datasets 
annTestFile='{}/annotations/COCO_amodal_val2014_with_classes_poly.json'.format(dataDir)
imgTestFile = '{}/val2014'.format(dataDir)
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_coco_val", {}, annTestFile, imgTestFile)


In [ ]:
dataset_dicts = DatasetCatalog.get("amodal_coco_train")
annos = [img['annotations'] for img in dataset_dicts ]
invisible_mask = [ obj['invisible_mask'] for img in annos for obj in img if obj.get('invisible_mask')]
visible_mask = [ obj['visible_mask'] for img in annos for obj in img ]
print(invisible_mask[2])
[len(i) for i in invisible_mask]

## Sanity checks : overfitting small datasets 

In [ ]:
# Register small datasets for debugging 
dataset_dicts = DatasetCatalog.get("amodal_coco_train")
import random
imgs = random.sample(dataset_dicts,k=10)
# imgs = dataset_dicts[:10]
DatasetCatalog.register("small_amodal_test", lambda : imgs)
metadata = {}
MetadataCatalog.get("small_amodal_test").set(
        image_root="datasets/coco/train2014", evaluator_type="coco", **metadata
    )


In [ ]:
from pycocotools.coco import COCO
coco_api = COCO(annTrainFile)
cat_ids = sorted(coco_api.getCatIds())
cats = coco_api.loadCats(cat_ids)
# The categories in a custom json file may not be sorted.
thing_classes = [c["name"] for c in sorted(cats, key=lambda x: x["id"])]

In [ ]:
metadata = {}
MetadataCatalog.get("small_amodal_test").set(thing_classes = thing_classes, **metadata )
# MetadataCatalog.get("small_amodal_test")

In [ ]:
smalldicts = DatasetCatalog.get("small_amodal_test")
for i in range(len(smalldicts)):
    im = cv2.imread(smalldicts[i]["file_name"])
    imshow(im[:, :, ::-1])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(smalldicts[i])
    imshow(vis.get_image()[:, :, ::-1])

### Training 
- remove orcnnEvaluation directory everytime : create json_file for small dict

In [ ]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("small_amodal_test",)
cfg.DATASETS.TEST = ("small_amodal_test",)
cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0015  # pick a good LR
cfg.SOLVER.STEPS = (1300,1400)
cfg.SOLVER.MAX_ITER = 500 
cfg.VIS_PERIOD = 20
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.OUTPUT_DIR = "orcnnCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
import pdb; pdb.set_trace()
trainer.train()

### Tensorboard

In [ ]:
!tensorboard --logdir=orcnnCheckpoint --port=6006

### Evaluation

In [ ]:
# 
# [556865, 14]
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("small_amodal_test", cfg, False, output_dir="orcnnEvaluation")
val_loader = build_detection_test_loader(cfg, "small_amodal_test")
# import pdb;pdb.set_trace()
inference_on_dataset(trainer.model, val_loader, evaluator) 

### Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("small_amodal_test",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("small_amodal_test")
# for d in random.sample(dataset_dicts, 2): 
for i,d in enumerate(dataset_dicts):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("small_amodal_test"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d,'invisible_mask')
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("small_amodal_test"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"),'pred_invisible_masks')
    imshow(v.get_image()[:, :, ::-1])
    cv2.imwrite('SmallSamplesVisualizing/Sample{}.png'.format(i), vis.get_image()[:, :, ::-1]) 
    cv2.imwrite('SmallSamplesVisualizing/Test{}.png'.format(i), v.get_image()[:, :, ::-1]) 

## Training from a COCO-pretrained model as provided by Detectron2
1. Finetuning : For the case of COCOA amodal the final output layers that are class-specific had to be initialized randomly as the number of classes and their semantic meaning did not fit to the number of classes of COCO.

## Training 

In [ ]:
import os
import numpy as np
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.SOLVER.BASE_LR = 0.0015  # pick a good LR
cfg.SOLVER.STEPS = (1400,)
cfg.SOLVER.MAX_ITER = 1500 
cfg.VIS_PERIOD = 500
cfg.OUTPUT_DIR = "orcnnCheckpoint"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set the testing threshold for this model
import pdb; pdb.set_trace()
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

--Return--
> <ipython-input-3-395c2bcdba3c>(20)<module>()->None
-> import pdb; pdb.set_trace()
(Pdb) c
[03/01 23:14:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), 

[03/01 23:14:03 d2.data.datasets.coco]: Loaded 2276 images in COCO format from datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json
[03/01 23:14:03 d2.data.build]: Removed 0 images with no usable annotations. 2276 images left.
[03/01 23:14:03 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 2311         |   bicycle    | 21           |      car      | 279          |
|  motorcycle   | 47           |   airplane   | 65           |      bus      | 92           |
|     train     | 60           |    truck     | 115          |     boat      | 79           |
| traffic light | 13           | fire hydrant | 35           |   stop sign   | 12           |
| parking meter | 10           |    bench     | 48           |     bird      | 57           |
|      cat    

[03/01 23:15:47 d2.utils.events]: eta: 0:05:46  iter: 339  total_loss: 1.083  loss_cls: 0.164  loss_box_reg: 0.205  loss_amodal_mask: 0.232  loss_visible_mask: 0.198  loss_invisible_mask: 0.143  loss_rpn_cls: 0.010  loss_rpn_loc: 0.019  time: 0.2997  data_time: 0.0037  lr: 0.000509  max_mem: 3020M
[03/01 23:15:53 d2.utils.events]: eta: 0:05:41  iter: 359  total_loss: 1.101  loss_cls: 0.190  loss_box_reg: 0.182  loss_amodal_mask: 0.237  loss_visible_mask: 0.225  loss_invisible_mask: 0.164  loss_rpn_cls: 0.009  loss_rpn_loc: 0.016  time: 0.3003  data_time: 0.0036  lr: 0.000539  max_mem: 3031M
[03/01 23:15:59 d2.utils.events]: eta: 0:05:34  iter: 379  total_loss: 0.940  loss_cls: 0.167  loss_box_reg: 0.224  loss_amodal_mask: 0.211  loss_visible_mask: 0.188  loss_invisible_mask: 0.151  loss_rpn_cls: 0.010  loss_rpn_loc: 0.012  time: 0.3003  data_time: 0.0035  lr: 0.000569  max_mem: 3031M
[03/01 23:16:05 d2.utils.events]: eta: 0:05:30  iter: 399  total_loss: 1.037  loss_cls: 0.173  loss_box

[03/01 23:18:34 d2.utils.events]: eta: 0:03:09  iter: 879  total_loss: 1.095  loss_cls: 0.226  loss_box_reg: 0.227  loss_amodal_mask: 0.221  loss_visible_mask: 0.172  loss_invisible_mask: 0.119  loss_rpn_cls: 0.009  loss_rpn_loc: 0.013  time: 0.3046  data_time: 0.0038  lr: 0.001319  max_mem: 3100M
[03/01 23:18:40 d2.utils.events]: eta: 0:03:02  iter: 899  total_loss: 0.741  loss_cls: 0.160  loss_box_reg: 0.169  loss_amodal_mask: 0.172  loss_visible_mask: 0.170  loss_invisible_mask: 0.061  loss_rpn_cls: 0.008  loss_rpn_loc: 0.007  time: 0.3044  data_time: 0.0039  lr: 0.001349  max_mem: 3100M
[03/01 23:18:46 d2.utils.events]: eta: 0:02:56  iter: 919  total_loss: 1.402  loss_cls: 0.329  loss_box_reg: 0.262  loss_amodal_mask: 0.244  loss_visible_mask: 0.219  loss_invisible_mask: 0.141  loss_rpn_cls: 0.011  loss_rpn_loc: 0.014  time: 0.3046  data_time: 0.0038  lr: 0.001379  max_mem: 3100M
[03/01 23:18:52 d2.utils.events]: eta: 0:02:50  iter: 939  total_loss: 1.098  loss_cls: 0.240  loss_box

In [ ]:
!tensorboard --logdir=orcnnCheckpoint --port=6006

## Evalution

In [ ]:
cfg.DATASETS.TEST = ("amodal_coco_val",)
cfg.DATASETS.TRAIN = ("amodal_coco_train",)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
#  evaluate its performance using AP metric implemented in COCO API.
from detectron2.evaluation import AmodalEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = AmodalEvaluator("amodal_coco_val", cfg, False, output_dir="orcnnEvaluation")
val_loader = build_detection_test_loader(cfg, "amodal_coco_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

## Visualization

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
from detectron2.data import DatasetCatalog
from detectron2.utils.visualizer import Visualizer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_orcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "orcnnCheckpoint"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set the testing threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("amodal_coco_val",)
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get("amodal_coco_val")
for d in random.sample(dataset_dicts, 1):    
    im = cv2.imread(d["file_name"])
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get("amodal_coco_val"), scale=0.5)
    vis = visualizer.draw_dataset_dict(d,"invisible_mask")
    imshow(vis.get_image()[:, :, ::-1])
#     import pdb;pdb.set_trace()
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("amodal_coco_val"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"),'pred_invisible_masks')
    imshow(v.get_image()[:, :, ::-1])